In [109]:
import os
import pandas as pd
from pandas import DataFrame
import numpy as np

In [110]:
import pandas as pd

# CSV 파일 경로 (이미지 만들어줄 csv파일의 경로를 넣어주세요!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!)
csv_file_path ='./swinT최대(0.6553).csv'
# CSV 파일을 데이터프레임으로 불러오기
dataframe = pd.read_csv(csv_file_path)
#앙상블 결과를 그릴떄에는 중간에 코드 한줄이 다름 (csv파일 형식이 좀 달라서)

# 데이터프레임 확인
print(dataframe)
#데이터프레임을 리스트로
df_list = dataframe.values.tolist()
newlist=[0]*len(df_list)
print(df_list[3][0])
print(type(df_list[1][0]))
for i in range(len(df_list)):
    #앙상블 안한 일반 모델일떄 
    newlist[i]=str(df_list[i][0]).split(' ')
    #앙상블 한 결과일때 (앙상블 결과면 위에꺼 주석처리하고 밑에꺼 주석해제)!!!!!!!!!!!!!!!!!!!!!!
    #newlist[i]=str(df_list[i][1]).split(' ')
    newlist[i].pop()
reallist=list([[] for _ in range(10)] for idx in range(len(df_list)))
#print(reallist)
for i,temp in enumerate(newlist):
    for j in range(0,len(temp),6):   
        reallist[i][int(temp[j])].append([float(x) for x in temp[j+1:j+6]]) #[float(x) for x in a]    append(temp[j+1:j+6])
    

                                       PredictionString       image_id
0     0 0.094164036 473.1461 556.53516 558.7289 591....  test/0000.jpg
1     3 0.5278075 344.90585 243.8132 755.0635 696.19...  test/0001.jpg
2     0 0.9000873 879.67224 466.05133 1022.0185 630....  test/0002.jpg
3     9 0.9947879 156.05684 263.59015 911.9555 812.0...  test/0003.jpg
4     0 0.19444406 411.32422 400.38705 657.9198 573....  test/0004.jpg
...                                                 ...            ...
4866  0 0.6686456 293.25284 346.89975 669.64667 690....  test/4866.jpg
4867  0 0.9198968 629.68054 666.003 775.4372 722.428...  test/4867.jpg
4868  0 0.17426667 544.76636 508.56384 722.8225 800....  test/4868.jpg
4869  0 0.99836004 4.368046 224.47337 1017.9979 932....  test/4869.jpg
4870  0 0.14799047 241.8742 821.79517 290.6881 854.4...  test/4870.jpg

[4871 rows x 2 columns]
9 0.9947879 156.05684 263.59015 911.9555 812.063 9 0.806569 29.460182 65.87979 989.43384 940.96643 9 0.14509879 1.4558204 5

In [111]:
#output 시각화 (원래 없던 코드 내가 만든거)
#output[i][j] > i번째 이미지에 j번째 클래스 박스들 모음 [x1(왼쪽상단x좌표),y1(왼쪽상단),x2(오른쪽하단),y2,클래스에 속할 확률]
#현재는 전체 이미지에 박스를 씌운 이미지를 생성하는 코드


#이미지 저장할 폴더 생성
#생성할 폴더명 입력!!!!!!!!!!!!!!!!!!!
folder_name='swinT최대(0.6553)'
os.mkdir(folder_name)

import cv2
class_names=["General trash","Paper","Paper pack","Metal","Glass","Plastic","Styrofoam","Plastic bag","Battery","Clothing"]
for i,image_info in enumerate(reallist):
    if i==100: #4천장여러개 만들면 서버가 힘들어해서 0번부터 100개만 만듬
        break
    #image_id = image_info['id']
    #test dataset일때랑 train data일때는 'test'를 'train'으로 변경
    #숫자를 4자리 문자열로 바꿔줌 ( 3 >'0003')
    stridx = str(i).zfill(4)
    #test이미지가 있는 경로 수정하기 !!!!!! 
    image_path = os.path.join('./dataset/images/test',stridx+'.jpg')
    image = cv2.imread(image_path)
    #boxes에는 박스 좌표를, class_ids는 클래스 ids를 저장
    boxes=[]
    class_ids=[]
    for idx in range(10):
        for tempidx in range(len(reallist[i][idx])):
            boxes.append(reallist[i][idx][tempidx][1:5])
            class_ids.append(idx)
           

    # 클래스 id와 색상 매핑
    class_colors = {
    0: (0, 255, 0),   # 클래스0에 대한 색상 (초록색)
    1: (0, 0, 255),   # 클래스1에 대한 색상 (빨간색)
    2: (255, 0, 0),   # 클래스2에 대한 색상 (파란색)
    3: (255, 255, 0), # 클래스3에 대한 색상 (노란색)
    4: (255, 0, 255), # 클래스4에 대한 색상 (자홍색)
    5: (0, 255, 255), # 클래스5에 대한 색상 (청록색)
    6: (128, 128, 0), # 클래스6에 대한 색상 (올리브색)
    7: (128, 0, 128), # 클래스7에 대한 색상 (보라색)
    8: (0, 128, 128), # 클래스8에 대한 색상 (청록색)
    9: (128, 128, 128)# 클래스9에 대한 색상 (회색)
}

    # 박스 시각화
    for i, box in enumerate(boxes):
        x1, y1, x2, y2 = box
        class_id = class_ids[i]
        color = class_colors[class_id]
        class_name = class_names[class_id]
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 1
        thickness = 2
        cv2.rectangle(image, (int(x1), int(y1)), (int(x2), int(y2)), color, thickness)
        cv2.putText(image, class_name, (int(x1), int(y1)-5), font, font_scale, color, thickness)
        if i==99:
            break
    
    # 결과 이미지 저장 (output 폴더안에 생성됨 inferenceoutput폴더 외에 다른 폴더에 만들고싶으면 ouput을 폴더명으로 바꺼주면댐)
    #현재 경로에서 yolov8x4863 폴더를 만들어준 다음에 안에 yolo_output_0001.jpg 이런식으로 데이터가 생성되게 됨
    output_path = os.path.join(folder_name, folder_name+'_output_{}.jpg'.format(stridx))
    cv2.imwrite(output_path, image)